In [1]:
import logging

import hydra

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [2]:
with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config", overrides={"dataset.generate_csv=True"})
dataset = hydra.utils.instantiate(cfg.dataset)

INFO:erc.utils:Instantiate KEMDy19 Dataset
INFO:erc.utils:./data/kemdy19.csv does not exists. Process from raw data


Processing ECG / EDA / Label from data/KEMDy19:   0%|          | 0/20 [00:00<?, ?it/s]

INFO:erc.utils:New dataframe saved as data/kemdy19.csv


In [3]:
cfg.dataset.mode = "valid"
cfg.dataset.generate_csv = False
for fold in range(0, 5):
    cfg.dataset.validation_fold = fold
    dataset = hydra.utils.instantiate(cfg.dataset)
    print(f"{fold}: {len(dataset)}")

INFO:erc.utils:Instantiate KEMDy19 Dataset
INFO:erc.utils:Instantiate KEMDy19 Dataset
INFO:erc.utils:Instantiate KEMDy19 Dataset
INFO:erc.utils:Instantiate KEMDy19 Dataset
INFO:erc.utils:Instantiate KEMDy19 Dataset


0: 1905
1: 2055
2: 2110
3: 2032
4: 2026


In [3]:
dl = hydra.utils.instantiate(cfg.dataloader)
batch = next(iter(dl))

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing ECG / EDA / Label:   0%|          | 0/20 [00:00<?, ?it/s]

In [4]:
from pathlib import Path

segment_id = dataset.total_df.segmend_id.iloc[250]
session, script_type, speaker = segment_id.split("_")

prefix = Path(dataset.wav_txt_path_fmt.format(session[-2:], script_type))
wav_path = prefix / f"{segment_id}.wav"
txt_path = prefix / f"{segment_id}.txt"

eda_path = dataset.eda_path_fmt.format(session[-2:], speaker[0])